In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split

df = pd.read_csv(data_csv, parse_dates=True, index_col=0)
df = df.dropna()

df["High_Low_per"] = (df["High"] - df["Close"]) / df["Close"] * 100
df["Per_change"] = (df["Open"] - df["Open"]) / df["Close"] * 100
df = df[["Adj Close", "High_Low_per", "Per_change", "Volume"]]
label_col = "Adj Close"
forecast_ceil = int(math.ceil(0.35 * len(df)))
df2 = df.copy()
df2["label"] = df2[label_col]
print(df2.__len__())

df["label"] = df[label_col].shift(-forecast_ceil)
# feaures X, labels Y
X = np.array(df.drop(["label"], 1))
X = preprocessing.scale(X)

X = X[:-forecast_ceil:]
print("X len", X.__len__())
X_lately = X[-forecast_ceil:]
print("X_lately len", X_lately.__len__())
df.dropna(inplace=True)
y = np.array(df["label"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = svm.SVR(kernel="rbf")
clf.fit(X_train, y_train) 
accuracy = clf.score(X_test, y_test)
print(y_test.shape, X_test.shape)

print(accuracy)
forecast_set = clf.predict(X_lately)
# 2. Integrate Predictions into DataFrame
df['Forecast'] = np.nan
last_date = df.index[-1]
for i in forecast_set:
    last_date += pd.Timedelta(days=1)
    df.loc[last_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
# 3. Plotting
plt.figure(figsize=(10,6))
plt.plot(df.index, df2['Adj Close'], label='Adj Close')
plt.plot(df.index, df['Adj Close'], label='Adj Close')
plt.plot(df.index, df['Forecast'], label='Forecast')
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('AAPL Stock Price Prediction using SVM')
plt.show()

    # forecast_set = clf.predict(X_lately)

    # # 2. Integrate Predictions into DataFrame
    # df["Forecast"] = np.nan
    # last_date = df.iloc[-1].name
    # last_unix = last_date.timestamp()
    # one_day = 86400
    # next_unix = last_unix + one_day
    # print(len(forecast_set))
    
    # for i in forecast_set:
    #     next_date = datetime.datetime.fromtimestamp(next_unix)
    #     next_unix += one_day
    #     df.loc[next_date] = [np.nan for _ in range(len(df.columns) - 1)] + [i]

    # # 3. Plotting
    # plt.figure(figsize=(10, 6))
    # df2["Adj Close"].plot()
    # df["Adj Close"].plot()
    # df["Forecast"].plot()
    # print(df.head())
    # plt.legend(loc=4)
    # # use index as x, not date
    

    # plt.ylabel("Price")
    # plt.show()

    # from matplotlib.pyplot import figure
    # figure(num=None, figsize=(40, 20), dpi=160, facecolor='w', edgecolor='k')
    # df['Adj Close'].plot()
    # df['Forecast'].plot()
    # plt.legend(loc=4)
    # plt.xlabel('Date')
    # plt.ylabel('Price')
    # plt.show()